# First-Class Functions

A programming language is said to have first-class function if it treats functions as first-class citizens. 

A first-class citizen (sometimes called first-class objects) is an entity that supports all the operations generally available to other entities. For exaple, being passed as an argument, returned from a function, and assigned to a variable. 

In [1]:
def square(x):
    return x * x

f = square(5)
print(square)
print(f)

<function square at 0x00000172C7310D08>
25


You can make **f** take the function by removing the **( )**.

Note: Some people only remove the arguments instead of the parenthesis. 

In [2]:
f = square
print(square)
print(f)

<function square at 0x00000172C7310D08>
<function square at 0x00000172C7310D08>


This means that **square** is a first-class function, and we can use **f** as a square

In [3]:
print(f(5))
print(square(5))

25
25


If a function accepts functions as arguments, or returns a function, this is called **Higher Order Functions**

## Function with function as parameters

In [4]:
def my_map(func, arg_list):
    """
    Takes a function and a list as arguments
    and returns a new list
    """
    result = []
    for i in arg_list:
        result.append(func(i))
    return result
 
# Test it with square func
squares = my_map(square, [1, 2, 3, 4, 5, 6])
print(squares)

[1, 4, 9, 16, 25, 36]


In [5]:
def cube(x):
    return x * x * x

cubes = my_map(cube, [1, 2, 3, 4, 5, 6])
print(cubes)

[1, 8, 27, 64, 125, 216]


## Return a function from another function
Note: To call the return value, which is a **callable** object, use parenthesis. 

In [6]:
def logger(msg):
    def log_message():
        print("Log:", msg)
    # NO parenthesis
    return log_message
# test it
log_hi = logger("Hi!")
# Now run it
log_hi()

Log: Hi!


In [8]:
# another example
def html_tag(tag):
    
    def wrap_text(msg):
        print("<{0}>{1}</{0}>".format(tag, msg))
        
    return wrap_text
# Test it
print_h1 = html_tag("h1")
print_h1("Test Headline")

<h1>Test Headline</h1>


In [10]:
print_p= html_tag("p")
print_p("test paragraph")

<p>test paragraph</p>


### Local functions
That is a function defined inside other functions. Remember the **def** key word is used to define functions. It simply binds the body of the function to a name. Just like any other object. The **def** is execute at runtime. So functions are defined at run time. 

In [11]:
def func1():
    """Regular function"""
    x = 1
    y = 2
    return x + y

def func2():
    """Regular function"""
    def local_func():
        """Local function"""
        a = "hello"
        b = " world"
        return a + b
    x = 1
    y = 2
    return x + y
        

Local function are bound by the **Local Enclosed Global Built-in (LEGB)** rule. 

**Local Function** are NOT members of the containing function in any way. It is just a local name binding. 

* Useful for specilized, one-off functions
* Aid in code organization and readability
* Similar to Lambdas, but more general
    * may contain multiple expression
    * may contain statements

## Closures and Nested Scopes

Remember, a local function can be returned from a function to be used later. 

How the fucntion can relate to the variable that no longer exist? How can the local function operates without that enclosing scope?

The answer is: **Closures**. A closure essentially remembers the object from the enclosing scope that the local function needs. It then keeps them "alive" so that when the local function is executed they can still be used. 

This prevents the auto garbage collection from cleaning the memory. 

Python implements closures with the **dunder closure** method. 

Where is this useful?

## Function Factories

A common use of **closures** is in **so-called function factories**. These functions return another function where the other function are specilized in some way based on the arguments to the "factory". 


Note: see raise_to.py

## Nonlocal keyword
Remember you can use the **global** keyword to bing names to global variables. 

**nonlocal** introduces names from the enclosing namespace into the **local** namespace

# Function Decorators

At a high-level **decorators** are a way to modify or enhance existing functions in a nonintrusive and amintanable way. 

In Python a decorator is an object that is **implemented as callable** that **takes a callable** and **returns a callable** object. 

Think about it as a function that takes a function as argument and returns another function. 

> @my_decorator<br>
> def my_funcion():<br>
>    ...

* Replace, enhance, or modify existing functions
* Does not change the original function definition
* Calling code des not need to change
* Decorator mechanism uses the modified function's original name. 

see: escape_unicode.py

## Classes as decorators
Before we were passing a function as an argument to a function, but how are going to pass a function as argument to a class?

We want to decorate the same function, but now through a **Class Decorator**

In [13]:
def decorator_function(original_function):
    def wrapper_function(*args, **kwargs):
        print('Wrapper executed this before {}'.format(
            original_function.__name__ ))
        return original_function(*args, **kwargs)

    return wrapper_function


class Decorator_class(object):
    def __init__(self, original_function):
        self._original_function = original_function
    # Make it callable    
    def __call__(self, *args, **kwargs):
        print("Call method executed this before {}".format(
        self._original_function.__name__))
        return self._original_function(*args, **kwargs)

#@decorator_function  # function decorator
@Decorator_class   # class decorator
def display():
    print("Display function ran")


#@decorator_function # function decorator
@Decorator_class   # class decorator
def display_info(name, age):
    print("Display_info ran with parameters ({}{})".format(
        name, age ))
    
# test it
display()
display_info("Mario", 12)

Call method executed this before display
Display function ran
Call method executed this before display_info
Display_info ran with parameters (Mario12)


Some people like to decorate with classes. I will be using functions from now on. 

In [17]:
def my_logger(orig_func):
    import logging
    logging.basicConfig(filename='{}.log'.format(
        orig_func.__name__), level = logging.INFO)
    def wrapper(*args, **kwargs):
        logging.info(
        "Ran with args: {}".format(args, kwargs))
        return orig_func(*args, **kwargs) 
    return wrapper

# Decorating
@my_logger
def display_info(name, age):
    print("Display_info ran with parameters ({} {})".format(
        name, age ))
# Test it
display_info("Mary", 33)

Display_info ran with parameters (Mary 33)


Another example is to time how long function run

In [21]:
import time

def my_timer(orig_func):
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = orig_func(*args, **kwargs)
        t2 = time.time() - t1
        print("{} ran in: {}".format(orig_func.__name__, t2))
        return result
    return wrapper
# Decorating
@my_timer
def display_info(name, age):
    print("Display_info ran with parameters ({} {})".format(
        name, age ))
    time.sleep(1) # sleep
    
# Test it
display_info("Mary", 33)

Display_info ran with parameters (Mary 33)
display_info ran in: 1.0107953548431396


## Multiple decorators
One more example to wrap everything. Apply both logger and timer decorators

In [22]:
# Decorating
@my_timer
@my_logger
def display_info(name, age):
    print("Display_info ran with parameters ({} {})".format(
        name, age ))
    time.sleep(1) # sleep
    
# Test it
display_info("Mary", 33)

Display_info ran with parameters (Mary 33)
wrapper ran in: 1.0022706985473633


It is always a good idea to preserve your original decorator information by using the **functools** module. All we need to do is to decorate all of the wrapper with the **wraps** function

In [23]:
# Decorating
@my_logger
@my_timer
def display_info(name, age):
    print("Display_info ran with parameters ({} {})".format(
        name, age ))
    time.sleep(1) # sleep
    
# Test it
display_info("Mary", 33)

Display_info ran with parameters (Mary 33)
display_info ran in: 1.0059611797332764


In [26]:
import time
from functools import wraps

def my_logger(orig_func):
    import logging
    logging.basicConfig(filename='{}.log'.format(
        orig_func.__name__), level = logging.INFO)
    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        logging.info(
        "Ran with args: {}".format(args, kwargs))
        return orig_func(*args, **kwargs) 
    return wrapper

def my_timer(orig_func):
    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = orig_func(*args, **kwargs)
        t2 = time.time() - t1
        print("{} ran in: {}".format(orig_func.__name__, t2))
        return result
    return wrapper

# Decorating
@my_logger
@my_timer
def display_info(name, age):
    print("Display_info ran with parameters ({} {})".format(
        name, age ))
    time.sleep(1) # sleep
    
# Test it
display_info("Tom", 33)

Display_info ran with parameters (Tom 33)
display_info ran in: 1.013390064239502
